In [1]:
import os
data_dir = os.path.join(os.getcwd(), "All_Data")

In [2]:
%pip install ffmpeg-python

You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.9/bin/python3.9 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [3]:
one_liner = "ffmpeg -i in.mp4 -vf select='between(n\,x\,y)' -vsync 0 image%d.png"
#get_num_frames_command = "ffprobe -v error -select_streams v:0 -count_frames -show_entries stream=nb_read_frames -of csv=p=0 input.mp4"

In [4]:
command_args = one_liner.split(' ')
#frames_command_args = get_num_frames_command.split(' ')
import subprocess

In [5]:
trial_ids = os.listdir(data_dir)
def process_dir_list(input_list):
    input_list.sort()
    if input_list[0] == ".DS_Store":
        del input_list[0]
    if input_list[-1] == ".DS_Store":
        del input_list[-1]
process_dir_list(trial_ids)

In [6]:
import numpy as np
import ffmpeg
trial_paths = list(map(lambda x: os.path.join(data_dir, x), trial_ids))
num_images = 10
for i in range(len(trial_paths)):
    trial_id = trial_ids[i]
    videos = os.listdir(trial_paths[i])
    process_dir_list(videos)
    video_paths = list(map(lambda x: os.path.join(trial_paths[i], x), videos))
    video_num_frames = []
    for j in range(len(videos)):
        metadata = ffmpeg.probe(video_paths[j])
        for stream in metadata['streams']:
            if stream['codec_type'] == 'video':
                video_num_frames.append(stream['nb_frames'])
    min_frames = video_num_frames[0]
    print(min_frames)
    for j in range(1, len(videos)):
        min_frames = min(min_frames,video_num_frames[j])
    indices = list(map(lambda x: round(x), np.linspace(0.,float(min_frames),num_images+1).tolist()))
    for idx in indices[1:-1]:
        cur_command_args = command_args.copy()
        cur_dir = os.path.join(os.getcwd(), "Training_Data", trial_id+str(idx))
        if not os.path.exists(cur_dir):
            os.mkdir(cur_dir)
        for j in range(len(videos)):
            cur_command_args = command_args.copy()
            cur_command_args[2] = video_paths[j]
            cur_command_args[-1] = os.path.join(cur_dir, videos[j].split('.')[0] + "-" + cur_command_args[-1])
            cur_command_args[4] = "select='between(n\,"+str(idx)+"\,"+str(idx)+")"
            subprocess.run(cur_command_args, stdout = subprocess.DEVNULL, stderr = subprocess.DEVNULL)

9612
9261
9201
9243
9201
9201
4260
4236
4299
4269
3918
3888
3909
3942
3864
2898
2988
